this file stores all the neural predictions on all 8k images for all models in `20200218` script

In [1]:
from sys import path
from os.path import relpath, realpath, abspath, join, exists, dirname
from os import makedirs
from itertools import product
from functools import partial
import numpy as np

In [2]:
from torchnetjson.builder import build_net

In [3]:
from thesis_v2 import dir_dict
from thesis_v2.data import load_data_helper
from thesis_v2.training.training_aux import load_training_results
from thesis_v2.training_extra.misc import count_params
from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import load_modules

load_modules()

In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200218 as param_iterator_obj,
    keygen
)

In [5]:
from thesis_v2.training_extra.data import generate_datasets
from thesis_v2.training_extra.evaluation import eval_fn_wrapper as eval_fn_wrapper_neural
from thesis_v2.training.training import eval_wrapper
from thesis_v2.data.prepared.yuanyuan_8k import get_data, get_neural_data

In [6]:
from torch.backends import cudnn
import torch
cudnn.enabled = True
cudnn.deterministic = True
cudnn.benchmark = False

_data_cache = dict()

def get_data_cached(input_size, seed):
    if (input_size, seed) not in _data_cache:
        _data_cache[input_size, seed] = get_data('a', 200, input_size,
                                                 ('042318', '043018', '051018'),
                                                 scale=0.5,
                                                 seed=seed)
    return _data_cache[input_size, seed]

In [7]:
from scipy.stats import pearsonr

In [8]:
def load_everything():
    dir_to_save = join(dir_dict['analyses'], 'responses_yuanyuan_8k_a_3day+maskcnn_polished_with_rcnn_k_bl+20200218')
    makedirs(dir_to_save, exist_ok=True)
    old_case = 0
    new_case = 0
    for idx, param in enumerate(param_iterator_obj().generate()):
        
        assert len(param) == 22
        
        assert param['kernel_size_l1'] == 9
        assert param['ff_1st_block']
        
        
        
        assert param['split_seed'] == 'legacy'
        assert param['pooling_ksize'] == 3
        assert param['pooling_type'] == 'avg'
        assert not param['bn_after_fc']
        assert param['rcnn_bl_psize'] == 1
        assert param['rcnn_bl_ptype'] is None
        assert param['input_size'] == 50
        
#         if not (param['act_fn'] == 'relu' and param['out_channel']==16 and param['rcnn_bl_cls']==4 and param['kernel_size_l23']==3):
#             continue
        
        
        # 12 to go
        # remove smoothness_name and scale_name, 10 to go
        
#         assert param['model_seed'] == 0
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        
        if idx % 10 == 0:
            print(idx)
            print(key)
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
        result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        num_epochs = [len(x) for x in result['stats_all']]
        
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
    
        
        # ok. time to load data and get response.
        
        datasets = get_data_cached(param['input_size'], param['split_seed'])


        datasets = {
            'X_train': datasets[0].astype(np.float32),
            'y_train': datasets[1],
            'X_val': datasets[2].astype(np.float32),
            'y_val': datasets[3],
            'X_test': datasets[4].astype(np.float32),
            'y_test': datasets[5],
        }

        # only the test one is needed.
        datasets_all = generate_datasets(
            **datasets,
            per_epoch_train=False,
            shuffle_train=False,
        )
        shape_dict = {
            'train': 5120,
            'val': 1280,
            'test': 1600,
        }
        
        for subset in ('train', 'val', 'test'):
            
            file_to_store_this = join(dir_to_save, key, subset + '.npy')
            if exists(file_to_store_this):
                old_case += 1
                continue
                
            new_case += 1
                
            makedirs(dirname(file_to_store_this), exist_ok=True)
        
            result_on_the_go = eval_wrapper(result['model'].cuda(),
                                            datasets_all[subset],
                                            'cuda',
                                            1,
                                            partial(eval_fn_wrapper_neural, loss_type=param['loss_type'], yhat_reduce_axis=1),
                                            (lambda dummy1,dummy2,dummy3: torch.tensor(0.0)),
                                            return_responses=True
                                           )
            
            ret = np.asarray(result_on_the_go['corr'])
            
            # store responses, only last timestamp
            responses = np.concatenate([x[0] for x in result_on_the_go['responses']], axis=1)[-1]
            
            assert responses.shape == (shape_dict[subset], 79)
            
            # store the responses.
            np.save(file_to_store_this, responses)
        
            if subset in {'test', 'val'}:
                assert abs(result['stats_best']['stats'][subset]['corr_mean'] - result_on_the_go['corr_mean']) < 1e-5
                assert abs(result['stats_best']['stats'][subset]['corr_mean'] - ret.mean()) < 1e-5
                
                # another check.
                # make sure order of data aligns.
                
                label = datasets[f'y_{subset}']
                assert responses.shape == label.shape
                assert responses.ndim == 2
                assert responses.shape[1] == 79
                corr_each = np.array(
                    [pearsonr(yhat, y)[0] if np.std(yhat) > 1e-5 and np.std(y) > 1e-5 else 0 for yhat, y in
                          zip(responses.T, label.T)])
                corr_each_ref = np.asarray(result_on_the_go['corr'])
                assert corr_each.shape == corr_each_ref.shape
                # not equal because in `eval_fn_wrapper_neural`, the label is in float32, while here it's in float64.
                assert np.allclose(corr_each_ref, corr_each)
                

    print('new', new_case)
    print('old', old_case)
    

In [9]:
load_everything()

# no mean value change across time, which is expected given the loss function.

# seems that std value increases across time.
# maybe higher std <-> more diverity and higher accuracy in prediction?

0
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c1/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
10
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lmse/m_se0
20
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lmse/m_se0
30
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lmse/m_se0
40
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg

370
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lpoisson/m_se0
380
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lpoisson/m_se0
390
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
400
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c1/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lmse/m_se0
410
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8

730
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lpoisson/m_se0
740
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lpoisson/m_se0
750
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lpoisson/m_se0
760
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c1/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lpoisson/m_se0
770
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selega

1090
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lpoisson/m_se1
1100
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
1110
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lpoisson/m_se1
1120
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c1/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lpoisson/m_se1
1130
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch3

1450
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
1460
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lpoisson/m_se1
1470
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lpoisson/m_se1
1480
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l2/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c1/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
1490
yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k